In [1]:
import os
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

# --- הגדרות נתיבים ---
# הנתיב לתיקייה שסיפקת
folder_path = r"C:\Users\elnatan_u\Downloads\drive-download-20251221T143827Z-1-001"
# שם התיקייה שבה יישמר בסיס הנתונים הוקטורי
db_path = "vectorstore_db"

def build_or_load_vectorstore():
    # שימוש במודל ה-Embedding מה-Notebook המקורי שלך
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # בדיקה אם כבר סרקנו את הקבצים בעבר
    if os.path.exists(db_path):
        print("טוען בסיס נתונים קיים מהדיסק...")
        return FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    
    print("מתחיל לסרוק קבצי PDF בתיקייה (זה עשוי לקחת זמן)...")
    all_chunks = []
    
    # מעבר על כל הקבצים בתיקייה
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            full_path = os.path.join(folder_path, filename)
            print(f"מעבד את: {filename}")
            
            try:
                with pdfplumber.open(full_path) as pdf:
                    text = ""
                    for page in pdf.pages:
                        page_content = page.extract_text()
                        if page_content:
                            text += page_content + "\n"
                
                # פיצול הטקסט לקטעים קטנים (Chunks) לחיפוש מדויק
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000, 
                    chunk_overlap=150
                )
                chunks = text_splitter.split_text(text)
                
                # שמירת המקור לכל קטע
                for chunk in chunks:
                    all_chunks.append(f"Source: {filename}\nContent: {chunk}")
            except Exception as e:
                print(f"שגיאה בעיבוד הקובץ {filename}: {e}")

    if not all_chunks:
        raise ValueError("לא נמצא טקסט לחילוץ בקבצי ה-PDF בתיקייה שצוינה.")

    # יצירת האינדקס ושמירה לדיסק
    vectorstore = FAISS.from_texts(all_chunks, embeddings)
    vectorstore.save_local(db_path)
    print(f"בסיס הנתונים נוצר ונשמר בתיקייה: {db_path}")
    return vectorstore

# --- אתחול המערכת ---
vector_db = build_or_load_vectorstore()

def ask_question(query):
    # 1. חיפוש 4 הקטעים הרלוונטיים ביותר
    docs = vector_db.similarity_search(query, k=4)
    context = "\n\n---\n\n".join([doc.page_content for doc in docs])
    
    # 2. יצירת תשובה בעזרת המודל
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    prompt = f"""
    ענה על השאלה בצורה מקצועית על סמך ההקשר המצורף בלבד.
    בסוף התשובה, ציין בפירוט מאילו קבצי מקור נלקח המידע.

    Context:
    {context}

    Question:
    {query}
    
    Answer:"""
    
    response = llm.invoke(prompt)
    return response.content

# --- הרצה לדוגמה ---
if __name__ == "__main__":
    print("\nהצ'אט-בוט מוכן!")
    question = "מה המסקנות העיקריות לגבי הטיפול באינסולין והחקיקה בנושא?"
    print(f"\nשאלה: {question}")
    print(f"\nתשובה:\n{ask_question(question)}")

מתחיל לסרוק קבצי PDF בתיקייה (זה עשוי לקחת זמן)...
מעבד את: 1. Baumrind S, Frantz RC_ The reliability of head film measurements. 1. landmark identification.pdf
מעבד את: 10. Jarabak Analysis, in Enlow DH. Facial Growth. Ch 14.pdf
מעבד את: 100. Vaden JL_  Diagnosis of the vertical dimentsion 2002.pdf
מעבד את: 101. Feres MFN, Abreu L et al_Effectiveness of open bite correction when managing deleterious oral habits.pdf
מעבד את: 102. Brezniak N, Arad A, Heller M, Dinte A, Wasserstein A_ Pathognomonic cephalometric characteristics of Angle Class II Division 2 malocclusion.pdf
מעבד את: 103. George SM., Campbell PhM., Tadlock LP., Schneiderman E, Buschang PH. Keys to Class II  correction_ A comparison of 2 extraction protocols.pdf
מעבד את: 104. Jacobson A_  A key to the understanding of extraoral forces Jacobson AJODO 1979.pdf
מעבד את: 105. Hershey HG, Houghton CW, Burstone J_ Unilateral face-bows A theoretical and laboratory analysis  1981.pdf
מעבד את: 106. Baumrind S, Molthen R, West EE, Mil

Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P4' is an invalid float value
Cannot set gray non-stroke color because /'P5' is an invalid float value
Cannot set gray non-stroke color because /'P6' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value


מעבד את: 163. Zachrisson BU_ On Excellence in Finishing – Part 2 -  1986.pdf
מעבד את: 164. Kokich VG_Excellence in finishing modifications for the perio restorative patient Kokich Semin Orthod 2003.pdf
מעבד את: 165. Shalish M, Peck S, Wasserstein A, Peck L_ Increased occurrence of dental anomalies associated with infraocclusion of deciduous molars.pdf
מעבד את: 167. Czochorowska E, Zachrisson BU_Outcome of tooth transplantation_survival and success rates 17-41 years posttreatment.pdf
מעבד את: 168. Thilander B, Odman J, Lekholm U_ Orthodontic aspects of the use of oral implants in adolescents a 10 year follow-up studyr EJO 2001.pdf
מעבד את: 169. Kokich VO, Kinzer GA_ Managing congenitally missing lateral incisors. Part I Canine substitution_ J Esthet Restor Dent 2005.pdf
מעבד את: 17. Leslie LR, Southard et al_ Prediction of mandibular growith rotation_ Assessment of the Skieller, Bjork and Linde-Hansen method.pdf
מעבד את: 170. Kokich VO Kinzer GA_ Managing Congenitally Missing Lateral In

In [1]:
import os
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# --- הגדרות ---
# וודא שה-API KEY מוגדר במערכת או הוסף אותו כאן:
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

folder_path = r"C:\Users\elnatan_u\Downloads\drive-download-20251221T143827Z-1-001"
db_path = "vectorstore_db"

def build_or_load_vectorstore():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    if os.path.exists(db_path):
        print("💡 טוען בסיס נתונים קיים מהדיסק...")
        return FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    
    print("📂 סורק קבצי PDF בתיקייה... זה עשוי לקחת זמן.")
    docs = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            full_path = os.path.join(folder_path, filename)
            print(f"📄 מעבד את: {filename}")
            
            try:
                with pdfplumber.open(full_path) as pdf:
                    text = ""
                    for page in pdf.pages:
                        page_content = page.extract_text()
                        if page_content:
                            text += page_content + "\n"
                
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
                chunks = text_splitter.split_text(text)
                
                for chunk in chunks:
                    docs.append(Document(page_content=chunk, metadata={"source": filename}))
                    
            except Exception as e:
                print(f"⚠️ שגיאה ב-{filename}: {e}")

    if not docs:
        raise ValueError("לא נמצא טקסט בתיקייה.")

    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(db_path)
    print(f"✅ בסיס הנתונים נשמר ב: {db_path}")
    return vectorstore

def ask_question(query, vector_db):
    docs = vector_db.similarity_search(query, k=4)
    
    # בניית הקשר עם ציון המקור לכל קטע
    context_parts = []
    for doc in docs:
        source = doc.metadata.get("source", "Unknown")
        context_parts.append(f"[מקור: {source}]\n{doc.page_content}")
    
    context = "\n\n---\n\n".join(context_parts)
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    prompt = f"""
    אתה עוזר מקצועי המשיב על שאלות בהתבסס על המידע המצורף בלבד.
    ענה בעברית רהוטה. 
    בסוף התשובה, ציין בפירוט את שמות קבצי המקור ששימשו אותך.

    Context:
    {context}

    Question:
    {query}
    
    Answer:"""
    
    response = llm.invoke(prompt)
    return response.content

# --- הרצת הצ'אט ---
if __name__ == "__main__":
    vector_db = build_or_load_vectorstore()
    
    print("\n" + "="*30)
    print("🤖 הצ'אט מוכן! הקלד 'יציאה' כדי לסיים.")
    print("="*30)
    
    while True:
        user_input = input("\n❓ שאל שאלה: ")
        
        if user_input.lower() in ["יציאה", "exit", "quit", "ביי"]:
            print("להתראות!")
            break
            
        if not user_input.strip():
            continue
            
        print("\n⏳ חושב...")
        try:
            answer = ask_question(user_input, vector_db)
            print(f"\n✨ תשובה:\n{answer}")
        except Exception as e:
            print(f"❌ שגיאה: {e}")

💡 טוען בסיס נתונים קיים מהדיסק...

🤖 הצ'אט מוכן! הקלד 'יציאה' כדי לסיים.
להתראות!


In [2]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [4]:
import streamlit as st
import os
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# --- הגדרות API ---
# החלף את ה-API_KEY במפתח האמיתי שלך
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# --- הגדרות דף ---
st.set_page_config(page_title="עוזר ה-PDF החכם", page_icon="📚", layout="centered")

# עיצוב בסיסי לעברית
st.markdown("""
    <style>
    .stApp {
        direction: RTL;
        text-align: right;
    }
    </style>
    """, unsafe_allow_html=True)

st.title("📚 צ'אט עם מסמכי ה-PDF שלך")
st.subheader("מערכת RAG לחיפוש וניתוח מסמכים")

# --- הגדרות נתיבים ---
folder_path = r"C:\Users\elnatan_u\Downloads\drive-download-20251221T143827Z-1-001"
db_path = "vectorstore_db"

# פונקציה לטעינה או בנייה של בסיס הנתונים
@st.cache_resource
def get_vector_db():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # אם בסיס הנתונים כבר קיים, נטען אותו
    if os.path.exists(db_path):
        return FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    
    # אם לא קיים, נסרוק את התיקייה
    if not os.path.exists(folder_path):
        st.error(f"התיקייה לא נמצאה: {folder_path}")
        st.stop()
        
    docs = []
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    
    if not pdf_files:
        st.error("לא נמצאו קבצי PDF בתיקייה המצוינת.")
        st.stop()

    progress_bar = st.progress(0)
    status_text = st.empty()

    for i, filename in enumerate(pdf_files):
        status_text.text(f"מעבד קובץ {i+1} מתוך {len(pdf_files)}: {filename}")
        full_path = os.path.join(folder_path, filename)
        try:
            with pdfplumber.open(full_path) as pdf:
                text = ""
                for page in pdf.pages:
                    page_content = page.extract_text()
                    if page_content:
                        text += page_content + "\n"
            
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
            chunks = text_splitter.split_text(text)
            
            for chunk in chunks:
                docs.append(Document(page_content=chunk, metadata={"source": filename}))
        except Exception as e:
            st.warning(f"שגיאה בעיבוד {filename}: {e}")
        
        progress_bar.progress((i + 1) / len(pdf_files))

    if not docs:
        st.error("לא חולץ טקסט מהקבצים.")
        st.stop()

    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(db_path)
    status_text.text("✅ בסיס הנתונים מוכן!")
    return vectorstore

# אתחול המערכת
with st.spinner("מאתחל מערכת..."):
    vector_db = get_vector_db()

# --- ניהול שיחה ---
if "messages" not in st.session_state:
    st.session_state.messages = []

# הצגת היסטוריית ההודעות
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# קלט מהמשתמש
if prompt := st.chat_input("שאל שאלה על המסמכים שלך..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        with st.spinner("מחפש תשובה במקורות..."):
            # שליפה מהווקטורים
            docs = vector_db.similarity_search(prompt, k=4)
            
            # בניית הקשר עם מקורות
            context_list = []
            sources = set()
            for d in docs:
                src = d.metadata.get('source', 'לא ידוע')
                sources.add(src)
                context_list.append(f"--- תוכן ממקור ({src}) ---\n{d.page_content}")
            
            context = "\n\n".join(context_list)
            
            # יצירת תשובה עם GPT
            llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
            
            full_prompt = f"""
            ענה על השאלה בצורה מקצועית ומפורטת בעברית, בהתבסס אך ורק על ה-Context המצורף.
            אם התשובה לא נמצאת בטקסט, אמור שאינך יודע.
            בסוף התשובה, רשום את רשימת הקבצים המדויקת ששימשה אותך.

            Context:
            {context}

            Question:
            {prompt}
            """
            
            try:
                response = llm.invoke(full_prompt).content
                st.markdown(response)
                st.session_state.messages.append({"role": "assistant", "content": response})
            except Exception as e:
                st.error(f"שגיאה בפנייה ל-OpenAI: {e}")

2025-12-22 15:41:18.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:41:18.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [5]:
import os
from dotenv import load_dotenv # אם מותקן לך python-dotenv
load_dotenv()

False